![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

## Cheatsheet

### Our assumptions

1. Mode of transmission of the disease from person to person is through contact ("contact transmission") between a person who interacts with an infectious person. 
    
2. Once a person comes into contact with the pathogen, there is a period of time (called the latency period) in which they are infected, but cannot infect others (yet!). 

3. Population is not-constant (that is, people are born and die as time goes by).

4. A person in the population is either one of:
    - <b>S</b>usceptible, i.e. not infected but not yet exposed, 
    - <b>E</b>xposed to the infection, i.e. exposed to the virus, but not yet infectious, 
    - <b>I</b>nfectious, and 
    - <b>R</b>ecovered from the infection. 
    
5. People can die by "natural causes" during any of the stages. We assume an additional cause of death associated with the infectious stage. 

### Our flow diagram

How does a person move from one stage into another? In other words, how does a person go from susceptible to exposed, to infected, to recovered? 

$\Delta$: Per-capita birth rate.

$\mu$: Per-capita natural death rate.

$\alpha$: Virus-induced average fatality rate.

$\beta$: Probability of disease transmission per contact (dimensionless) times the number of contacts per unit time.

$\epsilon$: Rate of progression from exposed to infectious (the reciprocal is the incubation period).

$\gamma$: Recovery rate of infectious individuals (the reciprocal is the infectious period).

$$\stackrel{\Delta N} {\longrightarrow} \text{S} \stackrel{\beta\frac{S}{N} I}{\longrightarrow} \text{E} \stackrel{\epsilon}{\longrightarrow} \text{I}  \stackrel{\gamma}{\longrightarrow} \text{R}$$
$$\hspace{1.1cm} \downarrow \mu \hspace{0.6cm} \downarrow \mu  \hspace{0.5cm} \downarrow \mu, \alpha  \hspace{0.1cm} \downarrow \mu $$

### Our system of equations

$N$ is updated at each time step, and infected peopel die at a higher rate. 

$$ N = S + E + I + R$$

We can then express our model using differential equations

$$\frac{dS}{dt} = \Delta N - \beta \frac{S}{N}I - \mu S$$

$$\frac{dE}{dt} = \beta \frac{S}{N}I - (\mu + \epsilon )E$$

$$\frac{dI}{dt} = \epsilon E - (\gamma+ \mu + \alpha )I$$

$$\frac{dR}{dt} = \gamma I - \mu R$$


Also, we can keep track of the dead people, dead due to the infection. 

$$\frac{dD}{dt} = \alpha I $$

### Our code

When we translate from Math to Python, it is useful to give appropriate names to our variables. 

|Math symbol|Variable name in Python | What it represents|
| - |  - | - |
$\Delta $ |$\text{Delta}$| Per-capita birth rate
|$\mu$|$\text{mu}$|Per-capita natural death rate
|$\alpha$|$\text{alpha}$| Virus-induced average fatality rate.
|$\beta$|$\text{beta}$|Probability of disease transmission per contact (dimensionless) times the number of contacts per unit time.
|$\epsilon$|$\text{epsilon}$|Rate of progression from exposed to infectious (the reciprocal is the incubation period).
|$\gamma$|$\text{gamma}$|Recovery rate of infectious individuals (the reciprocal is the infectious period).


In [ ]:
import numpy as np
from scipy.integrate import odeint
import pandas as pd
import plotly.express as px 
import plotly.graph_objects as go

# The SEIR model differential equations.
def deriv(y, t, Delta, beta, mu, epsilon,gamma,alpha):
    S, E, I, R, D = y
    N = S + E + I + R
    dS = Delta*N  - beta*S*I/N - mu*S
    dE = beta*S*I/N - (mu + epsilon)*E
    dI = epsilon*E - (gamma + mu + alpha)*I
    dR = gamma*I - mu*R
    dD = alpha*I 
    
    return [dS,dE, dI, dR, dD]

# Solving the system

# A grid of time points (in days)
t = np.linspace(0, 750, 750)
# Initial number of infected and recovered individuals, I0 and R0.
S0, E0,I0, R0 ,D0 = 37000000,0,100,0,0
# Total population, N.
N = S0 + E0 + I0 + R0
# Initial conditions vector
y0 = S0,E0, I0, R0, D0
Delta = 0 # natural birth rate. Set to zero for simplicity
mu = 0 # natural death rate. Set to zero for simplicity
alpha = 0.005  # death rate due to disease
beta = 0.9 # an interaction parameter. Rate for susceptible to exposed. 
epsilon = 0.1 # rate from exposed to infectious
gamma = 0.5 # rate from infectious to recovered (We expect this to be bigger than mu)
numerator = beta*epsilon
denominator = (alpha + gamma + mu)*(epsilon + mu)
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(Delta, beta, mu, epsilon,gamma,alpha))
S, E,I, R, D = ret.T

S,E,I,R,D = np.ceil(S),np.ceil(E),np.ceil(I),np.ceil(R),np.ceil(D)

seir_simulation = pd.DataFrame({"Susceptible":S,"Exposed":E,
                                "Infectious":I,"Recovered":R,"Deaths":D, "Time (days)":t})

px.line(seir_simulation,"Time (days)",'Infectious',title="Number of infectious people")

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)